In [1]:
import base64
import numpy as np
# from pycocotools import _mask as coco_mask
import typing as t
import zlib
import torch
import shutil
import os
import pandas as pd
import tifffile as tiff
import matplotlib.pyplot as plt
from pathlib import Path
from glob import glob
from collections import defaultdict
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from IPython.display import Image as show_image
from torch.utils.data import DataLoader
from skimage.measure import regionprops_table, label, regionprops
from pycocotools import _mask as coco_mask
import gc
import warnings
import torchvision.transforms as T
import segmentation_models_pytorch as smp
from PIL import Image
import torch.nn as nn
import random
import cv2
warnings.filterwarnings("ignore")

In [2]:
class model_config:
    current_fold = 0
    key = "BT" #"MoCoV2"
    pretrained_resnet = False
    seed = 42
    train_batch_size = 8
    valid_batch_size = 16
    epochs = 200  # ~15 minutes per epoch
    learning_rate = 0.0014 # 0.001 for bs=16
    scheduler = "CosineAnnealingLR"
    num_training_samples = 5499
    T_max = int(
        num_training_samples / train_batch_size * epochs)  # number of iterations for a full cycle, need to change for different # of iterations. (iteration = batch size)
    weight_decay = 1e-6  # explore different weight decay (Adam optimizer)
    n_accumulate = 1
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    iters_to_accumulate = max(1, 32 // train_batch_size)  # for scaling accumulated gradients
    eta_min = 1e-5
    model_save_directory = os.path.join(os.getcwd(), "model",
                                        "seresnext_attention_dropout_dilation_v2_retry")

# sets the seed of the entire notebook so results are the same every time we run for reproducibility. no randomness, everything is controlled.
def set_seed(seed=42):
    np.random.seed(seed)  #numpy specific random
    random.seed(seed)  # python specific random (also for albumentation augmentations)
    torch.manual_seed(seed)  # torch specific random
    torch.cuda.manual_seed(seed)  # cuda specific random
    # when running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False  # when deterministic = true, benchmark = False, otherwise might not be deterministic
    os.environ['PYTHONHASHSEED'] = str(seed)  # set a fixed value for the hash seed, for hases like dictionary

set_seed(model_config.seed)

In [3]:
class HubmapDataset(torch.utils.data.Dataset):
    def __init__(self, imgs, transforms):
        self.transforms = transforms
        self.imgs = imgs
        self.name_indices = [os.path.splitext(os.path.basename(i))[0] for i in imgs]

    def __getitem__(self, idx):
        # load images and masks
        img_path = self.imgs[idx]
        name = self.name_indices[idx]
        array = tiff.imread(img_path)
        img = Image.fromarray(array)
        img = self.transforms(img)
        return img, name

    def __len__(self):
        return len(self.imgs)

In [9]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
debug = True
model_path = r"C:\Users\labuser\hubmap\unet++_resnet50\model\seresnext_attention_dropout_dilation_v3\best_epoch-00.pt"
base_dir = Path('/kaggle/input/hubmap-hacking-the-human-vasculature')
image_paths = r"\\fatherserverdw\Kevin\hubmap\train_overlap\images"
test_paths = [os.path.join(image_paths,x) for x in os.listdir(image_paths) if x.endswith(".tif")][0:10] #load all train and debug or do just first hundreds
test_paths

['\\\\fatherserverdw\\Kevin\\hubmap\\train_overlap\\images\\5631a47d5b0c.tif',
 '\\\\fatherserverdw\\Kevin\\hubmap\\train_overlap\\images\\5631a47d5b0c_bot_right.tif',
 '\\\\fatherserverdw\\Kevin\\hubmap\\train_overlap\\images\\2fd7649afbc1.tif',
 '\\\\fatherserverdw\\Kevin\\hubmap\\train_overlap\\images\\2fd7649afbc1_top_right.tif',
 '\\\\fatherserverdw\\Kevin\\hubmap\\train_overlap\\images\\88c95fb9fb14.tif',
 '\\\\fatherserverdw\\Kevin\\hubmap\\train_overlap\\images\\88c95fb9fb14_bot_right.tif',
 '\\\\fatherserverdw\\Kevin\\hubmap\\train_overlap\\images\\dd690b7d9a47.tif',
 '\\\\fatherserverdw\\Kevin\\hubmap\\train_overlap\\images\\dd690b7d9a47_top_right.tif',
 '\\\\fatherserverdw\\Kevin\\hubmap\\train_overlap\\images\\dd690b7d9a47_bot_right.tif',
 '\\\\fatherserverdw\\Kevin\\hubmap\\train_overlap\\images\\5566406c59ee.tif']

In [10]:
test_transforms = T.Compose([T.PILToTensor(),T.ConvertImageDtype(torch.float32)]) # Size C x H x W tensor with float dtype

dataset_test = HubmapDataset(test_paths, transforms = test_transforms)
test_dl = torch.utils.data.DataLoader(
        dataset_test, batch_size=1, shuffle=False, num_workers=0, pin_memory=True)

In [11]:
def build_model():
    model = smp.UnetPlusPlus(encoder_name="se_resnext50_32x4d", encoder_weights=None, encoder_depth = 5, decoder_channels = [512, 256, 128, 64, 32], activation='sigmoid',
                             in_channels=3, classes=1, decoder_attention_type="scse", decoder_use_batchnorm=True,
                             aux_params={"classes": 1, "pooling": "max", "dropout": 0.5})
    model.to(device)  # model to gpu
    return model

In [15]:
model = build_model()
id_list, heights, widths, prediction_strings = [],[],[],[]
with torch.no_grad():
    model.load_state_dict(torch.load(model_path))
    for img, idx in tqdm(test_dl):
        model.eval() #eval stage
        img = img.to(device)
        prediction, _  = model(img)
        # prob_prediction = nn.Sigmoid()(prediction)
        prediction = torch.squeeze(prediction)
#         print("prob_prediction is {}".format(prob_prediction))
        print("max of prob prediction is {}".format(torch.max(prediction)))
        print("mean of prob prediction is {}".format(torch.mean(prediction)))
        print("min of prob prediction is: {}".format(torch.min(prediction)))

 20%|██        | 2/10 [00:00<00:01,  7.78it/s]

max of prob prediction is 0.029573699459433556
mean of prob prediction is 8.378188067581505e-05
min of prob prediction is: 4.66595811303705e-05
max of prob prediction is 0.029575519263744354
mean of prob prediction is 8.379008795600384e-05
min of prob prediction is: 4.667925168178044e-05


 40%|████      | 4/10 [00:00<00:00,  7.30it/s]

max of prob prediction is 0.029575254768133163
mean of prob prediction is 8.378141501452774e-05
min of prob prediction is: 4.667043685913086e-05
max of prob prediction is 0.029575519263744354
mean of prob prediction is 8.379008795600384e-05
min of prob prediction is: 4.667925168178044e-05


 60%|██████    | 6/10 [00:00<00:00,  7.03it/s]

max of prob prediction is 0.02957586571574211
mean of prob prediction is 8.37851403048262e-05
min of prob prediction is: 4.667435132432729e-05
max of prob prediction is 0.02956826239824295
mean of prob prediction is 8.380161307286471e-05
min of prob prediction is: 4.666954555432312e-05


 70%|███████   | 7/10 [00:00<00:00,  6.96it/s]

max of prob prediction is 0.02957264333963394
mean of prob prediction is 8.379553764825687e-05
min of prob prediction is: 4.667065877583809e-05
max of prob prediction is 0.02956826239824295
mean of prob prediction is 8.380161307286471e-05
min of prob prediction is: 4.666954555432312e-05
max of prob prediction is 0.02957719750702381
mean of prob prediction is 8.379673818126321e-05
min of prob prediction is: 4.667021130444482e-05


100%|██████████| 10/10 [00:01<00:00,  8.22it/s]

max of prob prediction is 0.029574550688266754
mean of prob prediction is 8.379123755730689e-05
min of prob prediction is: 4.6665674744872376e-05
